In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [2]:
TRAIN_PATH = 'data/data_train.csv'
TEST_PATH = 'data/data_test.csv'
FEATURES_PATH = 'data/features.csv'

In [3]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)
df_features = dd.read_csv(FEATURES_PATH, sep="\t")

In [4]:
features_train = df_features.loc[df_features['id'].isin(df_train['id'])].compute()
features_test = df_features.loc[df_features['id'].isin(df_test['id'])].compute()

In [5]:
del df_features

In [6]:
def merge_data(df,features):
    data_merged = pd.merge(df, features, on='id')
    
    duplicated_data = data_merged[data_merged['id'].duplicated(keep=False)].sort_values(by='id')
    duplicated_data['time_diff'] = abs(duplicated_data['buy_time_x'] - duplicated_data['buy_time_y'])
    duplicated_data = duplicated_data.sort_values(by = ['Unnamed: 0_x', 'time_diff'])
    
    before_clean = duplicated_data.index.tolist()
    after_clean = duplicated_data.drop_duplicates(subset=['Unnamed: 0_x', 'id'],keep='first').index.tolist()
    to_delete = list(set(before_clean) - set(after_clean))
    
    data_merged.drop(to_delete,axis=0,inplace=True)
    data_merged.drop(columns=['Unnamed: 0_x','Unnamed: 0_y', 'buy_time_y'], inplace=True)
    data_merged.rename(columns={"buy_time_x": "buy_time"}, inplace=True)
    
    del duplicated_data
    return data_merged

In [7]:
merged_train = merge_data(df_train,features_train)

In [8]:
merged_train.head()

,id,vas_id,buy_time,target,0,1,2,3,4,5,...,243,244,245,246,247,248,249,250,251,252
0,540968,8.0,1537131600,0.0,-31.559971,327.360888,-45.500786,274.753202,-50.918246,253.64821,...,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0
1,1454121,4.0,1531688400,0.0,547.270029,238.430888,533.329214,274.803202,527.911754,253.69821,...,-972.373846,-613.770792,-25.996269,-19.630448,-278.747724,-24.832889,-0.694428,-11.175933,-0.45614,0.0
2,2458816,1.0,1534107600,0.0,-92.139971,-95.469112,-106.080786,-139.596798,-111.498246,-142.90179,...,-977.373846,-613.770792,-25.996269,-37.630448,-304.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,3535012,5.0,1535922000,0.0,54.880029,12.970888,54.079214,-9.116798,48.661754,-30.22179,...,-977.373846,-613.770792,-25.996269,-18.630448,-133.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0
4,1693214,1.0,1535922000,0.0,45.160029,295.240888,64.679214,344.283202,59.261754,323.17821,...,-965.373846,-612.770792,-22.996269,-32.630448,-127.747724,-4.832889,-0.694428,-12.175933,-0.45614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860047,3812226,2.0,1546203600,0.0,29.750029,6.200888,24.279214,-37.936798,18.861754,-59.04179,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
860048,2480469,2.0,1546203600,0.0,-89.179971,-56.499112,-103.120786,-109.106798,-108.538246,-130.21179,...,-66.373846,-321.770792,-25.996269,62.369552,1167.252276,41.167111,-0.694428,54.824067,-0.45614,0.0
860049,158236,2.0,1546203600,0.0,-96.799971,62.140888,-110.740786,9.533202,-116.158246,-11.57179,...,-683.373846,-470.770792,-25.996269,-37.630448,99.252276,178.167111,-0.694428,191.824067,-0.45614,0.0
860050,1825525,2.0,1546203600,0.0,-96.799971,-81.919112,-110.740786,-128.596798,-116.158246,-76.82179,...,1018.626154,1367.229208,-24.996269,-35.630448,-237.747724,-21.832889,-0.694428,-8.175933,-0.45614,1.0


In [ ]:
merged_train.to_csv('data/train.csv')

In [ ]:
merged_test = merge_data(df_test,features_test)

In [ ]:
merged_test.head()

In [13]:
merged_test.to_csv('data/test.csv')